In [15]:
import matplotlib.pyplot as plt
from scipy.stats import binom
import numpy as np
import math
import scipy.stats as stats
# to use hedgehog, one needs to install two packages vose and hedgehog
# pip install git+https://github.com/MaxHalford/vose
# pip install git+https://github.com/MaxHalford/hedgehog
# you may also need to install graphviz to plot PGM
# conda install -c conda-forge python-graphviz 
import pandas as pd
from scipy.special import logsumexp
from IPython.display import Markdown as md
from random import randrange
def hide_code_in_slideshow():   
    from IPython import display
    import binascii
    import os
    uid = binascii.hexlify(os.urandom(8)).decode()    
    html = """<div id="%s"></div>
    <script type="text/javascript">
        $(function(){
            var p = $("#%s");
            if (p.length==0) return;
            while (!p.hasClass("cell")) {
                p=p.parent();
                if (p.prop("tagName") =="body") return;
            }
            var cell = p;
            cell.find(".input").addClass("hide-in-slideshow")
        });
    </script>""" % (uid, uid)
    display.display_html(html, raw=True)
#  a hack to hide code from cell: https://github.com/damianavila/RISE/issues/32    

In [16]:
%%html
<style>
 .container.slides .celltoolbar, .container.slides .hide-in-slideshow {
    display: None ! important;
}
    
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
th, td {
  padding: 5px;
}
th {
  text-align: left;
}
</style>

In [17]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

# Foreword 

The following code are used for this lecture
  * most of the code have been adapted from https://github.com/aimacode/aima-python

In [18]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    
def g(n): return n.path_cost    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]



def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

In [19]:
FIFOQueue = deque

LIFOQueue = list

StackAgenda = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

In [20]:
class RouteProblem(Problem):
    """A problem to find a route between locations on a `Map`.
    Create a problem with RouteProblem(start, goal, map=Map(...)}).
    States are the vertexes in the Map graph; actions are destination states."""
    
    def actions(self, state): 
        """The places neighboring `state`."""
        return self.map.neighbors[state]
    
    def result(self, state, action):
        """Go to the `action` place, if the map says that is possible."""
        return action if action in self.map.neighbors[state] else state
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""
        return self.map.distances[s, s1]
    
    def h(self, node):
        "Straight-line distance between state and the goal."
        locs = self.map.locations
        return straight_line_distance(locs[node.state], locs[self.goal])
    
    
def straight_line_distance(A, B):
    "Straight-line distance between two points."
    return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5

In [21]:
class Map:
    """A map of places in a 2D world: a graph with vertexes and links between them. 
    In `Map(links, locations)`, `links` can be either [(v1, v2)...] pairs, 
    or a {(v1, v2): distance...} dict. Optional `locations` can be {v1: (x, y)} 
    If `directed=False` then for every (v1, v2) link, we add a (v2, v1) link."""

    def __init__(self, links, locations=None, directed=False):
        if not hasattr(links, 'items'): # Distances are 1 by default
            links = {link: 1 for link in links}
        if not directed:
            for (v1, v2) in list(links):
                links[v2, v1] = links[v1, v2]
        self.distances = links
        self.neighbors = multimap(links)
        self.locations = locations or defaultdict(lambda: (0, 0))

        
def multimap(pairs) -> dict:
    "Given (key, val) pairs, make a dict of {key: [val,...]}."
    result = defaultdict(list)
    for key, val in pairs:
        result[key].append(val)
    return result

In [22]:
# Some specific RouteProblems

romania = Map(
    {('O', 'Z'):  71, ('O', 'S'): 151, ('A', 'Z'): 75, ('A', 'S'): 140, ('A', 'T'): 118, 
     ('L', 'T'): 111, ('L', 'M'):  70, ('D', 'M'): 75, ('C', 'D'): 120, ('C', 'R'): 146, 
     ('C', 'P'): 138, ('R', 'S'):  80, ('F', 'S'): 99, ('B', 'F'): 211, ('B', 'P'): 101, 
     ('B', 'G'):  90, ('B', 'U'):  85, ('H', 'U'): 98, ('E', 'H'):  86, ('U', 'V'): 142, 
     ('I', 'V'):  92, ('I', 'N'):  87, ('P', 'R'): 97},
    {'A': ( 76, 497), 'B': (400, 327), 'C': (246, 285), 'D': (160, 296), 'E': (558, 294), 
     'F': (285, 460), 'G': (368, 257), 'H': (548, 355), 'I': (488, 535), 'L': (162, 379),
     'M': (160, 343), 'N': (407, 561), 'O': (117, 580), 'P': (311, 372), 'R': (227, 412),
     'S': (187, 463), 'T': ( 83, 414), 'U': (471, 363), 'V': (535, 473), 'Z': (92, 539)})


r0 = RouteProblem('A', 'A', map=romania)
# r1 = RouteProblem('A', 'B', map=romania)
# r2 = RouteProblem('N', 'L', map=romania)
# r3 = RouteProblem('E', 'T', map=romania)
# r4 = RouteProblem('O', 'M', map=romania)

In [26]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """

    def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    
    def h1(self, node):
        """The misplaced tiles heuristic."""
        return hamming_distance(node.state, self.goal)
    
    def h2(self, node):
        """The Manhattan heuristic."""
        X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
        Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
        return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                   for (s, g) in zip(node.state, self.goal) if s != 0)
    
    def h(self, node): return self.h2(node)
    
    
def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b for a, b in zip(A, B))
    

def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')

class Board(defaultdict):
    empty = '.'
    off = '#'
    def __init__(self, board=None, width=8, height=8, to_move=None, **kwds):
        if board is not None:
            self.update(board)
            self.width, self.height = (board.width, board.height) 
        else:
            self.width, self.height = (width, height)
        self.to_move = to_move

    def __missing__(self, key):
        x, y = key
        if x < 0 or x >= self.width or y < 0 or y >= self.height:
            return self.off
        else:
            return self.empty
        
    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(row(y) for y in range(self.height))
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)
    
    
e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
e2 = EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0))
e3 = EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6))
e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))    

In [27]:
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)

        
def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(length=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose: report_counts(counts, str(p)[:40])
        report_counts(total_counts, 'TOTAL\n')
        
def report_counts(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} node extended|{:9,d} goal tested|{:5.0f} cost |{:8,d} length | {}'.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['length'], name))

In [37]:
# Python implementation of uniform cost search
def my_uniform_cost_search(problem, use_extendedList= True):
    agenda = PriorityQueue([Node(problem.initial)], key=g)
    #   extended list to keep what states have been extended
    extended_list = set()
    while agenda:
#       PQ.pop() returns the node with the least cost 
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                for child in expand(problem, node):
                    if not is_cycle(child):
#                       PriorityQueue taks care of this by inserting to 
#                       an appropriate location that is efficient for further operation
                        agenda.add(child)
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    agenda.add(child)
    return failure

In [36]:
def my_breadth_first_search(problem, use_extendedList = False):
    agenda = FIFOQueue([Node(problem.initial)])
#   extended list to keep what states have been extended
    extended_list = set()
    while agenda:
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                for child in expand(problem, node):
                    if not is_cycle(child):
                        agenda.appendleft(child)
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    agenda.appendleft(child)
    return failure

In [35]:
# Python implementation of best first search (based on heuristic only)
def my_best_first_search(problem, use_extendedList= False):
    agenda = PriorityQueue([Node(problem.initial)], key=problem.h)
    #   extended list to keep what states have been extended
    extended_list = set()
    while agenda:
#       PQ.pop() returns the node with the least cost 
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                for child in expand(problem, node):
                    if not is_cycle(child):
#                       PriorityQueue taks care of this by inserting to 
#                       an appropriate location that is efficient for further operation
                        agenda.add(child)
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    agenda.add(child)
    return failure

# CS5010 Artificial Intelligence Principles
### Lecture 19 A* search 


Lei Fang

University of St Andrews

# Last time

Informed searching algorithm variants
  * Greedy search or Best first search (informed version of UCS)
  * Hill climbing (informed version of DFS)
  * Beam search (informed version of BFS)

# Today

* A* search: an optimal informed search algorithm
  * a hybrid of UCS and Greedy search
  * where we make use of heuristics+costs

* Heuristic 
  * admissible heuristic guarantees optimal search result
  * qualities of heuristic matters

# Recap : Uniform Cost Search (UCS)


**Uniform Cost Searching** (UCS), the agenda reorganisation step is

<center><b>sort</b> the agenda based on the <b>costs</b> of the paths</center>
  
* denote cost function as $g$: given a path, $g(\cdot)$ returns the cost associated with the path

* e.g. for path [A,S,F]: $g(F) = \underbrace{140}_{A \text{ to } S} + \underbrace{99}_{S \text{ to } F}= 239$ 
* remember a node in a search tree represents a path !
  
**UCS** is also known as **Branch and Bound**

UCS pseudo-code:


<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
<font color="blue">0.</font>: initialize  <br> 
&nbsp; &nbsp; <font color="purple"><i>agenda</i></font> = [ [start] ]    &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;    # a list of paths or partial solutions  <br>
&nbsp; &nbsp; [<font color="purple"><i>extended_list</i></font> = {};]   &nbsp; &nbsp; &nbsp; &nbsp;    # optional: closed list of nodes   <br>     
&nbsp; &nbsp; <font color="green"><b>while</b></font> <font color="purple"><i>agenda</i></font> is not empty:<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">1.</font> <font color="purple"><i>path</i></font> = <font color="purple"><i>agenda</i></font>.pop(0)    &nbsp;&nbsp; # remove first element from agenda <br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">2.</font> <font color="green"><b>if</b></font> is-path-to-goal(<font color="purple"><i>path</i></font>, goal)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="green"><b>return</b></font> <font color="purple"><i>path</i></font><br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">3.</font> <font color="green"><b>otherwise</b></font> extend <font color="purple"><i>path</i></font> [if the ending node is not in <font color="purple"><i>extended_list</i></font>] <br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;[add the ending node to <font color="purple"><i>extended_list</i></font>]<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;<font color="green"><b>for each</b></font> connected node (child node of the ending node of <font color="purple"><i>path</i></font>)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;            make a <font color="purple"><i>new_path</i></font> that extends the connected node<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">4.</font> reject all <font color="purple"><i>new paths</i></font> from step <font color="blue">3</font> that contain <i>cycles</i><br> 
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">5.</font><b> add <font color="purple"><i>new paths</i></font> from step <font color="blue">4</font> to <font color="purple"><i>agenda</i></font> then sort the agenda based on the path costs</b><br>
&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;       # <b>yay ! another algorithm is "invented" </b> <br> 
<font color="blue">6.</font> <font color="green"><b>return</b></font> <font color="purple"><i>failure</i></font><br>  
</div>

# Recap: UCS walk through

Romania routing problem: find a route between Arad to Bucharest
<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romania.png" width = "900" height="300"/></center>  

<font color='blue'><b>Step 1</b></font>: after extending A

The next one to be popped and extended is **Z** (75)
* which is the cost of the path so far ! **no heuristic** is used


--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS1.png" width = "800" height="300"/></center>  

--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)
--------------------------

<font color='blue'><b>Step 2</b></font>: After extending **Z**

The next one to be popped, checked and extended becomes **T**: $g([A,T]) = 118$

--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS2.png" width = "800" height="300"/></center>  

--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)
--------------------------

<font color='blue'><b>Fast forward to Step 8</b></font>: after extending F


Note that one solution **[A,S,F,B]** with cost $g([A,S,F,B]) = 450$ has just been added to the agenda
  * which will not be returned immediately
  * why ? there might be other better options out there !
  * need to check them all before we return

--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS9.png" width = "800" height="300"/></center>

--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

<font color='blue'><b>Fast forward to the final return step</b></font>:

We are finally about to check **[A,S,R,P,B]** (418)
  * it passes the goal test and it will return as the solution
  * note that all other routes are officially worse $>418$
--------------------------
<font color='blue'><b>Tree view</b></font>: 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS11.png" width = "800" height="300"/></center>  


--------------------------
* <font color="salmon"><b> Salmon</b> </font> nodes: nodes in the agenda (open list)
* <font color="blue"><b> Blue</b> </font> nodes: nodes that have been popped and extended (close list)

# Recap: UCS is optimal 


<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS11_2.png" width = "800" height="300"/></center>  


* we are safe to stop here as all other are officially inferior
* what UCS has found is optimal

# How can we incorporate heuristic to our searching strategy

Note that UCS does not use heuristic at all (it is an uninformed search algorithm)
  * purely based on the path cost so far $g$
  * reorgnisation step: sort agenda based on $g$
  * which is optimal but expensive

We have also seen a heuristic only approach: i.e. **Greedy search** or Best first search
  * purely based on heuristic: we use straight line distance here
  * reorgnisation step: sort agenda based on $h_{SLD}$
  * which is not optimal but very fast 
  
  



Greedy search finds a solution within 3 steps!
* the number below each node represents the heuristic value, i.e. straight-line distance

--------------------------



|Greedy Search Tree view after 3 steps    |  Distance heuristic |   
|:-------------------------:|:-------------------------:|
|![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romBst3_.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/distances.png) | 

--------------------------

# A* search

**A* search** can be viewed as a hybrid of both, it makes use of both path cost so far $g(\cdot)$ and heuristic $h(\cdot)$

$$f(n) = g(n) + h(n)$$

* it considers both the cost so far: from the starting point to $n$
* plus the *expected* further cost $h(n)$: i.e. cost from $n$ to the goal !
* the sum $f(n)$ therefore represents some expected **cost to reach the goal via $n$**


  | | Complete| Optimal | agenda reorganisation
|:---|:---|:---|:---|
| Uniform Cost Search| Yes | Yes | add paths to agenda then sort the agenda based on $f(n) = g(n)$, or path cost|
| Greedy Search/BestFS| No | No |  add paths to agenda then sort the agenda based on $f(n) = h(n)$, or heuristic|
| A* search| ? | ?  | add paths to agenda then sort the agenda based on $f(n) = g(n) +h(n)$; the sum of path cost and heuristic|


$?$: it depends on the quality of heuristic, more on at the end

A* Search pseudo-code:


<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace;">
<font color="blue">0.</font>: initialize  <br> 
&nbsp; &nbsp; <font color="purple"><i>agenda</i></font> = [ [start] ]    &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;    # a list of paths or partial solutions  <br>
&nbsp; &nbsp; [<font color="purple"><i>extended_list</i></font> = {};]   &nbsp; &nbsp; &nbsp; &nbsp;    # optional: closed list of nodes   <br>     
&nbsp; &nbsp; <font color="green"><b>while</b></font> <font color="purple"><i>agenda</i></font> is not empty:<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">1.</font> <font color="purple"><i>path</i></font> = <font color="purple"><i>agenda</i></font>.pop(0)    &nbsp;&nbsp; # remove first element from agenda <br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">2.</font> <font color="green"><b>if</b></font> is-path-to-goal(<font color="purple"><i>path</i></font>, goal)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="green"><b>return</b></font> <font color="purple"><i>path</i></font><br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">3.</font> <font color="green"><b>otherwise</b></font> extend <font color="purple"><i>path</i></font> [if the ending node is not in <font color="purple"><i>extended_list</i></font>] <br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;[add the ending node to <font color="purple"><i>extended_list</i></font>]<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;<font color="green"><b>for each</b></font> connected node (child node of the ending node of <font color="purple"><i>path</i></font>)<br>
&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;            make a <font color="purple"><i>new_path</i></font> that extends the connected node<br>
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">4.</font> reject all <font color="purple"><i>new paths</i></font> from step <font color="blue">3</font> that contain <i>cycles</i><br> 
&nbsp; &nbsp; &nbsp; &nbsp;<font color="blue">5.</font><b> add <font color="purple"><i>new paths</i></font> from step <font color="blue">4</font> to <font color="purple"><i>agenda</i></font> then sort the agenda based on heuristic plus path costs or f(n)</b><br>
&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp;       # <b>yay ! another algorithm is "invented" </b> <br> 
<font color="blue">6.</font> <font color="green"><b>return</b></font> <font color="purple"><i>failure</i></font><br>  
</div>


In [49]:
# Python implementation of best first search (based on heuristic only)
def my_astar_search(problem, h = None, use_extendedList= False):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    agenda = PriorityQueue([Node(problem.initial)], key= lambda n: g(n)+h(n))
    #   extended list to keep what states have been extended
    extended_list = set() 
    while agenda:
#       PQ.pop() returns the node with the least cost 
        node = agenda.pop()
        if problem.is_goal(node.state):
            return node
        if use_extendedList:
            if node.state not in extended_list:
                extended_list.add(node.state)
                for child in expand(problem, node):
                    if not is_cycle(child):
#                       PriorityQueue taks care of this by inserting to 
#                       an appropriate location that is efficient for further operation
                        agenda.add(child)
        else:
            for child in expand(problem, node):
                if not is_cycle(child):
                    agenda.add(child)
    return failure

# A walk-through example of A* search

Skip initialisation step;

<font color='blue'><b>Step 1</b></font>:

Based on the path + heuristic, the next node to extend is Sibiu, or path [A,S]
  * its path cost is 140
  * the heuristic value of Sibiu is 253

It is the most promising option considering both the existing cost and "future cost"

--------------------------
<font color='blue'><b>Tree view</b></font>: 
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romAstar1.png)

--------------------------

<font color='blue'><b>Step 2</b></font>: after extending Sibiu

Based on the path + heuristic, the next node to extend is R, or path [A,S,R]
  * its path cost so far is 220
  * the heuristic value of R is 193: straight-line distance from R to Bucharest

It is the most promising option considering both the existing cost and "future cost"

--------------------------
<font color='blue'><b>Tree view</b></font>: 
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romAstar2.png)

--------------------------

<font color='blue'><b>Step 3</b></font>: after extending R

Based on the path + heuristic, the next node to extend is F, or path [A,S,F]
  * its path cost so far is 239
  * the heuristic value of F is 176: straight-line distance from F to Bucharest

It is the most promising option considering both the existing cost and "future cost"

--------------------------
<font color='blue'><b>Tree view</b></font>: 
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romAstar3.png)

--------------------------

<font color='blue'><b>Step 4</b></font>: after extending F

Note that one solution (to Bucharest) has been inserted to the agenda !
  * [A,S,F,B] with combined evaluation 450!
  * similar to UCS, it is not returned immediately, need to open the rest

Based on the path + heuristic, the next node to extend is P, or path [A,S,R,P]
  * its path cost so far is 317
  * the heuristic value of P is 100: straight-line distance from P to Bucharest

It is the most promising option considering both the existing cost and "future cost"

--------------------------
<font color='blue'><b>Tree view</b></font>: 
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romAstar4.png)

--------------------------

<font color='blue'><b>Step 5</b></font>: after extending P

Another (the optimal) solution (to Bucharest) has been inserted to the agenda !
  * [A,S,R,P,B] with combined evaluation 418!

Based on the path + heuristic, the next node to extend is B, or path [A,S,R,P,B], the **goal state** !
  * it will be returned as the solution!
  * also note all other routes are worse 
--------------------------
<font color='blue'><b>Tree view</b></font>: 
![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romAstar5_.png)

--------------------------

# Using heuristic greatly reduce the searching tree

Compare the three search trees at return

**A* search** returns the optimal result but has a smaller searching tree comparing with **UCS** 

  * therefore, more efficient in both space and time

--------------------------


| Greedy Search tree at return  |  UCS tree at return  |   A* search tree at return |
|:-------------------------:|:-------------------------:|:-------------------------:|
|![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romBst3_.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS11_3.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romAstar5_.png) | 

--------------------------

Thanks to heuristic, a lot of less optimal options are not added to agenda, e.g.[A,T] and [A,Z] branches
* you won't consider Aberdeen first if the destination is London !
* but you may eventually consider it if there is no other better options though 
  * e.g. all roads from Edinburgh/Glasgow to London have been blocked
  * then you may consider travelling via Aberdeen then fly to London

# Is A* optimal ?

It depends on the quality heuristic; if the heuristic is **admissible**, then **A* (without extension list) is optimal**


**Admissible** heuristic is **optimistic**: it always **under-estimates** the cost !
  * e.g. straight-line distance, $h_{SLD}$, the actual cost is always more expensive 
 
 
**Question** is $h(n) = 0$ for all nodes admissible for a route finding problem ? 

  * yes ! extremely optimistic, it believes it costs 0 from everywhere to the goal state
  * therefore, **UCS** can be viewed as special case of **A*** with 0 heuristic 
  
  $$f(n) = g(n) + 0$$

# Why under-estimated heuristics lead to optimality ?


--------------------------



|  UCS tree at return  |   A* search tree at return |
|:-------------------------:|:-------------------------:|
|  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romUCS11_3.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romAstar5_.png) | 

--------------------------

We already know UCS returns an optimal solution (check Lecture 17)


So the questions boils down to whether those routes **not considered** would affect the final result ?
  * e.g. [A,T,...],[A,Z,...], [A,S,O,...], etc


**No! considering them is a waste of time!**
* under-estimated heuristic gives us a lower bound of the cost via a node, which translates to 
  * the actual cost via n to the goal state is **at best** $f(n)$
* check the right hand side for what the lower bound means !

--------------------------


|  UCS tree   |   UCS tree really means |
|:-------------------------:|:-------------------------:|
|  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romAstar5_.png) |  ![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/romAstar5_2.png) | 

--------------------------

We are safe to stop here, all further routes are **officially worse**

In other words, A* is optimal if heuristic is admissible

# How about A* with extended_list ?


A* search algorithm with extended list is no longer optimal even the heuristic is admissible !

Check the following example. I will leave it as an exercise for you to verify. 

* first verify the heuristic is admissible 
* apply A* with extended list, what path it finds ? is that optimal ?
* such a heuristic is admissible but **inconsistent**, check its definition in AIAMA 

![](https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/inconsistent.png)

# Heuristic functions matters !

It should not surprise you that the quality of heuristic function matters !


For example, one possible heuristic for routing problem could be $h(n) = 0$
  * then we have UCS as a result 
  * which is complete and optimal
  * but leads to time and space complexity $O(b^{C^*/\epsilon})$

A rule of thumb: **good heuristic should be admissible but also reflect the true cost as much as possible!**

# 8 puzzle as an example 


<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/eightpuzzle.png" width = "600" height="400"/></center>

There are two possible heuristic functions being compared here
    
$h_1$: number of misplaced tiles
  * $h_1$ = 8 for the start state: all 8 tiles are out of position
  * $h_1$ is admissible, since it is obvious every tile needs to be moved at least once 

<center><img src="https://leo.host.cs.st-andrews.ac.uk/figs/figs4CS5010/searching/eightpuzzle.png" width = "600" height="400"/></center>

![](https://uploads-cdn.omnicalculator.com/images/manhattan_distance.png?width=850&enlarge=0&format=webp)  

$h_2$: Manhattan distance heuristic
  * $h_2 = \underbrace{3}_{\text{tile }1}+\underbrace{1}_{\text{tile }2}+\underbrace{2}_{\text{tile }3}+2+2+3+3+2=18$ 
  * it is also admissible: each tile at least needs this amount of moves
  
  
Clearly $h_2$ is of **better quality**: it is a closer estimate of the real cost  

# Demonstration 

In [50]:
def my_astar_misplaced_tiles(problem): return my_astar_search(problem, h=problem.h1)

report([my_uniform_cost_search, my_astar_misplaced_tiles, my_astar_search], 
       [e1, e2, e3, e4, e5])

my_uniform_cost_search:
      124 node extended|       46 goal tested|    5 cost |       5 length | EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8),
  222,942 node extended|  104,468 goal tested|   22 cost |      22 length | EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0),
  296,419 node extended|  147,241 goal tested|   23 cost |      23 length | EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6),
  457,038 node extended|  264,813 goal tested|   26 cost |      26 length | EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1),
  471,601 node extended|  280,892 goal tested|   27 cost |      27 length | EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1),
1,448,124 node extended|  797,460 goal tested|  103 cost |     103 length | TOTAL

my_astar_misplaced_tiles:
       17 node extended|        7 goal tested|    5 cost |       5 length | EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8),
   47,142 node extended|   17,559 goal tested|   22 cost |      22 length | EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0),
   90,785 node extended|   33,875 goal tested|  

As expected, $h_2$ is much faster and effective 
* it has extended and goal tested roughly 1/21 of nodes 


But informed searches (with both heuristic) in general perform better than uninformed search (UCS)
* also note all three are optimal !

# Summary

* A* search is optimal when adimissible heuristic is used
* Heuristic qualities matter